# 前言：
**从零开始摸索COVID19比赛奥义,错误地方请随时指出。**

我们是知识的搬运工，交流贴，目前还有很多不懂的，可随时交流。

In [ ]:
!conda install gdcm -c conda-forge -y

In [ ]:
! python -m pip install pillow

In [ ]:
import pandas as pd
import numpy as np
import cv2
import os
import re
import pydicom
import warnings
import glob
import ast
import math
import wandb
from PIL import Image
import albumentations as A
import torch
from matplotlib import pyplot as plt
from pydicom.pixel_data_handlers.util import apply_voi_lut
warnings.filterwarnings("ignore")

In [ ]:
def visualize(**images):
    """PLot images in one row."""
    n = len(images)
    plt.figure(figsize=(16, 5))
    for i, (name, image) in enumerate(images.items()):
        plt.subplot(1, n, i + 1)
        plt.xticks([])
        plt.yticks([])
        plt.title(' '.join(name.split('_')).title())
        plt.imshow(image)
    plt.show()

In [ ]:
DIR_INPUT = '../input/siim-covid19-detection/'
DIR_TRAIN = '../input/siim-covid19-detection/train/'
DIR_TEST = '../input/siim-covid19-detection/test/'

In [ ]:
#study numbers
len(os.listdir(DIR_TRAIN)),len(os.listdir(DIR_TEST))

测试结果中既包含study结果，也包含image结果。

In [ ]:
sample_sub = pd.read_csv(DIR_INPUT+'sample_submission.csv')
sample_sub = sample_sub.sort_values('id')
print(len(sample_sub))
sample_sub.head(5)

**study,series,image,patients**对应关系： 一个患者（patient）可以做多次检查（study），一次检查包含多个检查部位（series），而每个检查部位都有一张或多张相应的影像图像（image）（https://blog.csdn.net/qq_38330148/article/details/107094334） 

训练集：共6334个图片，6054次检查。有部分单次检查存在多个图片（多个部位的照片），参考例子：测试集中的0847751da0f7号检查。

In [ ]:
image_lel = pd.read_csv(DIR_INPUT+'train_image_level.csv')
image_lel = image_lel.sort_values('StudyInstanceUID')
print('image,',len(image_lel))
print('study,',len(image_lel['StudyInstanceUID'].unique()))
image_lel.head(5)

**图像角度研究：图像有无异常情况**

opacity: 肺内磨玻璃密度影。

肺内磨玻璃密度影(ground—glass opacity，GGO)是指高分辨率CT(high—resolution CT，HRCT)图像上表现为密度轻度增加，但其内的支气管血管束仍可显示，见于各种炎症、水肿、纤维化及肿瘤等病变。

In [ ]:
#训练集label信息。
label_type = dict()
for x in image_lel[['label']].iterrows():
    label = x[1].values[0].split(' ')[0]
    if label not in label_type:
        label_type[label] = 0
    label_type[label] += 1
label_type

**检查角度研究：犯了什么毛病。**

四类：Negative for Pneumonia，Typical Appearance，Indeterminate Appearance，Atypical Appearance

大概意思就是：没毛病，典型COVID19肺炎, 未确定的样式， 非典型COVID19肺炎样式

我的理解：opacity说明图片发现了问题， 一个或多个图片综合诊断判断病人是否存在：典型COVID19肺炎, 未确定的样式， 非典型COVID19肺炎样式。 是否如此还需进一步发掘。

In [ ]:
study_lel = pd.read_csv(DIR_INPUT+'train_study_level.csv')
study_lel = study_lel.sort_values('id')
print(len(study_lel))
study_lel.head(5)

# 样本详情：

Body Part Examined： 测试部位; Study Instance UID: Study编号； Series Instance UID： Series编号； SOP Instance UID： image编号； Patient's Sex： 患者性别； Patient ID： 患者编号


In [ ]:
sample_id = "../input/siim-covid19-detection/train/27f8616263ed/00eab0514ef6/7e4a4ef3ff9f.dcm"
dicom = pydicom.dcmread(sample_id)
dicom

In [ ]:
plt.imshow(dicom.pixel_array, cmap='bone')
print(dicom.pixel_array.shape) #图片尺寸

In [ ]:
image_lel['boxes'][2498],image_lel['label'][2498]#image label

In [ ]:
image_lel.head(2)

In [ ]:
study_lel['StudyInstanceUID'] = '-1'
def studylevel(x):
    x['StudyInstanceUID'] = x['id'].split('_')[0]
    return x

study_lel = study_lel.apply(lambda x:studylevel(x), axis = 1)
study_lel.head(2)

In [ ]:
#检查有没有重复的study号
flag = study_lel['StudyInstanceUID'].duplicated()
flag.any()

In [ ]:
del study_lel['id']
train_df = pd.merge(image_lel,study_lel,how='left',on='StudyInstanceUID')
train_df.head(5)

In [ ]:
for row in train_df.iterrows():
    if row[1][0].split('_')[0] == '7e4a4ef3ff9f':
        print(row)

一个生病的病人，不一定每张图片都能观察到症状。更有甚者，每张图片都没有问题，还被诊断有毛病。病人多次照片的原因，观察规律，大致判断为病人一次取图未检测到结果，重新再取图。

In [ ]:
#查看单检查多图片的情况
dup = train_df.groupby('StudyInstanceUID').count()>1
dup = dup[dup['id'] == True].index
repeat_df = train_df[train_df['StudyInstanceUID'].isin(dup)]
print('重复检查的图片数量：',len(repeat_df))
repeat_df.head(6)

code references: https://www.kaggle.com/andradaolteanu/siim-covid-19-box-detect-dcm-metadata

In [ ]:
def fix_inverted_radiograms(data, img):
    '''Fixes inverted radiograms - with PhotometricInterpretation == "MONOCHROME1"
    data: the .dcm dataset
    img: the .dcm pixel_array'''
    
    if data.PhotometricInterpretation == "MONOCHROME1":
        img = np.amax(img) - img
    
    img = img - np.min(img)
    img = img / np.max(img)
    img = (img * 255).astype(np.uint8)
    
    return img
def get_image_metadata(study_id, df):
    '''Returns the label and bounding boxes (if any)
    for a speciffic study id.'''
    
    data = df[df["StudyInstanceUID"] == study_id]
    
    if data["Negative for Pneumonia"].values == 1:
        label = "negative_for_pneumonia"
    elif data["Typical Appearance"].values == 1:
        label = "typical"
    elif data["Indeterminate Appearance"].values == 1:
        label = "indeterminate"
    else:
        label = "atypical"
        
    bbox = list(data["boxes"].values)
    
    return label, bbox
def return_coords(box):
    '''Returns coordinates from a bbox'''
    # Get the list of dictionaries
    box = ast.literal_eval(box)[0]
    # Get the exact x and y coordinates
    x1, y1, x2, y2 = box["x"], box["y"], box["x"] + box["width"], box["y"] + box["height"]
    # Save coordinates
    return (int(x1), int(y1), int(x2), int(y2))

In [ ]:
def show_dcm_info(study_ids, df):
    '''Show .dcm images along with description.'''
    wandb_logs = []
    
    fig, axes = plt.subplots(nrows=2, ncols=3, figsize=(21,10))

    # Get .dcm paths
    dcm_paths = [glob.glob(f"../input/siim-covid19-detection/train/{study_id}/*/*")[0]
                 for study_id in study_ids]
    datasets = [pydicom.dcmread(path) for path in dcm_paths]
    images = [apply_voi_lut(dataset.pixel_array, dataset) for dataset in datasets]

    # Loop through the information
    for study_id, data, img, i in zip(study_ids, datasets, images, range(2*3)):
        # Fix inverted images
        img = fix_inverted_radiograms(data, img)

        # Below function available in functions section ;)
        label, bbox = get_image_metadata(study_id, df)
        
        # Check for bounding box and add if it's the case
        try: 
            # For no bbox, the list is [nan]
            no_box = math.isnan(bbox[0])
            pass
        except TypeError:
            # Retrieve the bounding box
            all_coords = []
            for box in bbox:
                all_coords.append(return_coords(box))

            for (x1, y1, x2, y2) in all_coords:
                cv2.rectangle(img, (x1, y1), (x2, y2), (0, 0, 0), 15)
                cv2.putText(img, label, (x1, y1-14), 
                            cv2.FONT_HERSHEY_SIMPLEX, 3, (0, 0, 0), 4)
                
        # Plot the image
        x = i // 3
        y = i % 3
        
        axes[x, y].imshow(img, cmap="bone")
        axes[x, y].set_title(f"Label: {label} \n Sex: {data.PatientSex} | Body Part: {data.BodyPartExamined}", 
                  fontsize=14, weight='bold')
        axes[x, y].axis('off');
        
        # Save to W&B
        wandb_logs.append(wandb.Image(img, 
                                      caption=f"Label: {label} \n Sex: {data.PatientSex} | Body Part: {data.BodyPartExamined}"))

图像文件可视化观察

In [ ]:
# 典型症状的样本图示
show_dcm_info(study_ids=["72044bb44d41", "5b65a69885b6", "6aa32e76f998",
                         "c9ffe6312921", "082cafb03942", "d3e83031ebea"], 
              df=train_df)

In [ ]:
#非典型症状的样本图示
show_dcm_info(study_ids=["f807cd855d31", "8087e3bc0efe", "7249de10ed69",
                         "e300a4e86207", "4bac6c7da8b8", "f2d30ac37f7b"], 
              df=train_df)

In [ ]:
#不确定症状的样本图示
show_dcm_info(study_ids=["b949689a9ef1", "fe7e6015560d", "feffa20fac13",
                         "747483509d0e", "c70369caef91", "1e1b4b1b53cb"], 
              df=train_df)

In [ ]:
# 无肺炎症状的样本图示
show_dcm_info(study_ids=["612ea5194007", "db14e640e037", "d4ab797396b4",
                         "6ae8a88c4b0c", "b3cf474bee3b", "0ba55e5422ab"], 
              df=train_df)

# 数据集

In [ ]:
image_ids = train_df['id'].unique()
valid_ids = image_ids[-700:]
train_ids = image_ids[:-700]
len(train_ids),len(valid_ids)

In [ ]:
valid_df = train_df[train_df['id'].isin(valid_ids)]
train_df = train_df[train_df['id'].isin(train_ids)]
valid_df.head(2)

In [ ]:
from torch.utils.data import DataLoader, Dataset

data_link: thanks to https://www.kaggle.com/xhlulu/siim-covid19-resized-to-256px-jpg

In [ ]:
img = cv2.imread ('../input/siim-covid19-resized-to-256px-jpg/train/000a312787f2.jpg',1)
img.shape
visualize(
    image = img,
)

In [ ]:
meta = pd.read_csv('../input/siim-covid19-resized-to-256px-jpg/meta.csv')
meta.head()

meta_dict = dict()
for row in meta.iterrows():
    meta_dict[row[1][0]] = [row[1][1]/256,row[1][2]/256]
# meta_dict

In [ ]:
def img2tensor(img,dtype:np.dtype=np.float32): ## float 32/16
    if img.ndim==2 : img = np.expand_dims(img,2)
    img = np.transpose(img,(2,0,1))
    return torch.from_numpy(img.astype(dtype, copy=False))

def get_bbox(img_id, box):
    box_list= []
    b_list = box.split(' ')
    
    for i in range(len(box.split(' '))//6):
        box_list.append([float(b_list[i*6+2])/meta_dict[img_id][1], float(b_list[i*6+3])/meta_dict[img_id][0],
                         float(b_list[i*6+4])/meta_dict[img_id][1], float(b_list[i*6+5])/meta_dict[img_id][0]])
#         box_list.append([float(b_list[i*6+2])/meta_dict[img_id][0], float(b_list[i*6+3])/meta_dict[img_id][1],
#                          float(b_list[i*6+4])/meta_dict[img_id][0], float(b_list[i*6+5])/meta_dict[img_id][1]])
    return np.array(box_list).astype('float64')

def get_image(line):
    # LINE 4,5,6,7 -> NP，TA，IA，AA
    factors = line[2].split(' ')
    
    num = len(factors)//6
    
    return np.array([1 if factors[i*6]=='opacity' else 0 for i in range(num)])

#     return np.array([[line[4], line[5], line[6], line[7],] for i in range(num)]).astype('int64')
#     return np.array([i-4 for i in range(4,8) if line[i]==1 for j in range(num)]).astype('int64')


In [ ]:
train_df.values[0]

In [ ]:
class COVID19Dataset(Dataset):
    def __init__(self, df, pth, transforms=None):
        super().__init__()
        self.ids = df["id"].unique()
        self.df = df
        self.pth = pth
        self.transforms = transforms
        
    def __getitem__(self, index):
        img_id = self.ids[index].split('_')[0]
        img = cv2.imread (self.pth+img_id+'.jpg',1)
        
        line = self.df.values[index]
        boxes = get_bbox(img_id, line[2])
        image_labels = None
        image_confid = None
        image_labels = get_image(line)
        
        target = dict()
        target['boxes'] = torch.from_numpy(boxes)
        target['labels'] = torch.from_numpy(image_labels)
#         target['study_confid'] = torch.from_numpy(study_labels.astype('float32'))
        
        return img2tensor(img/255), target

    def __len__(self):
        return self.ids.shape[0]
        
        
train_dataset = COVID19Dataset(train_df,'../input/siim-covid19-resized-to-256px-jpg/train/', True)
# valid_dataset = COVID19Dataset(valid_df, None)

In [ ]:
x,y = train_dataset[0]
x,y

# 数据增强

# 模型

code references: https://www.kaggle.com/pestipeti/vinbigdata-fasterrcnn-pytorch-train/notebook

In [ ]:
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor

In [ ]:
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)

In [ ]:
num_classes = 2
in_features = model.roi_heads.box_predictor.cls_score.in_features
model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
device

In [ ]:
train_dataset = COVID19Dataset(train_df,'../input/siim-covid19-resized-to-256px-jpg/train/', True)
valid_dataset = COVID19Dataset(valid_df,'../input/siim-covid19-resized-to-256px-jpg/train/', True)
len(train_dataset), len(valid_dataset)

In [ ]:
def collate_fn(batch):
    return tuple(zip(*batch))

train_data_loader = DataLoader(
    train_dataset,
    batch_size=10,
    shuffle=True,
    num_workers=4,
    collate_fn=collate_fn
)

valid_data_loader = DataLoader(
    valid_dataset,
    batch_size=10,
    shuffle=False,
    num_workers=4,
    collate_fn=collate_fn
)

In [ ]:
# number = 7
# images, targets = next(iter(train_data_loader))
# images = list(image.to(device) for image in images)
# targets = [{k: v.to(device) for k, v in t.items()} for t in targets]
# boxes = targets[number]['boxes'].cpu().numpy().astype(np.int32)
# sample = images[number].permute(1,2,0).cpu().numpy()
# fig, ax = plt.subplots(1, 1, figsize=(16, 8))

# for box in boxes:
#     cv2.rectangle(sample,
#                   (box[0], box[1]),
#                   (box[2], box[3]),
#                   (0, 220, 0), 3)
    
# ax.set_axis_off()
# ax.imshow(sample)

# 训练

In [ ]:
class Averager:
    def __init__(self):
        self.current_total = 0.0
        self.iterations = 0.0

    def send(self, value):
        self.current_total += value
        self.iterations += 1

    @property
    def value(self):
        if self.iterations == 0:
            return 0
        else:
            return 1.0 * self.current_total / self.iterations

    def reset(self):
        self.current_total = 0.0
        self.iterations = 0.0

In [ ]:
model.to(device)
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=0.005, momentum=0.9, weight_decay=0.0005)
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=4, gamma=0.1)
num_epochs = 10

In [ ]:
loss_hist = Averager()
itr = 1

for epoch in range(num_epochs):
    loss_hist.reset()
    
    print(f'epoch {epoch} starting ...')
    for images, targets in train_data_loader:
        
        images_t = list(image.to(device) for image in images)
        targets_t = [{k: v.to(device) for k, v in t.items()} for t in targets]
        
        loss_dict = model(images_t, targets_t)
#         print(loss_dict)
        losses = sum(loss for loss in loss_dict.values())
        loss_value = losses.item()

        loss_hist.send(loss_value)

        optimizer.zero_grad()
        losses.backward()
        optimizer.step()
        if itr % 100 == 0:
            print(f"Iteration #{itr} loss: {loss_hist.value}")
        itr += 1
#         break
    
    # update the learning rate
    if lr_scheduler is not None:
        lr_scheduler.step()

    print(f"Epoch #{epoch} loss: {loss_hist.value}")
    print("Saving epoch's state...")
    torch.save(model.state_dict(), f"model_state_epoch_{epoch}.pth")
    break

In [ ]:
model.eval()
res = model([images_t[0]])
res

In [ ]:
targets_t[0]

# 预测结果

In [ ]:
TH = 0.6
def clean_res(res, th):
    for re in res:
        i = 0
        for i in range(len(re['scores'])):
            if re['scores'][i] < th:
                re['boxes'] = re['boxes'][:i]
                re['labels'] = re['labels'][:i]
                re['scores'] = re['scores'][:i]
                break
                
        if i == 0:
            re['boxes'] = torch.Tensor([[0,0,1,1]])
            re['labels'] = torch.Tensor([0])
            re['scores'] = torch.Tensor([1])
    return res

def record(res, row, test_name):
    PredictionString = ''
    for i in range(len(res['labels'])):
        if res['labels'][i] == 1:
            PredictionString += 'opacity '
            PredictionString += str(res['scores'][i].item()) + ' '
            
            for bx in range(len(res['boxes'][i])):
                if bx%2 == 0:
                    PredictionString += str(res['boxes'][i][bx].item()*meta_dict[test_name][1]) + ' '
                else:
                    PredictionString += str(res['boxes'][i][bx].item()*meta_dict[test_name][0]) + ' '
        else:
            PredictionString += 'none 1 0 0 1 1'
            break
    row['PredictionString'] = PredictionString
    return row
        
def predict(row):
    test_type = row['id'].split('_')[1]
    if test_type == 'study': return row
        
    test_name = row['id'].split('_')[0]
    img = cv2.imread (test_path+test_name+'.jpg',1)
    img = img2tensor(img/255)
    img = img.to(device)
    res = model([img])
    res = clean_res(res, TH)
    
    row = record(res[0], row, test_name)
    return row

In [ ]:
number = 0
images = [images_t[0].cpu()]
targets = [targets_t[0]]

images = list(image.to(device) for image in images)
targets = [{k: v.to(device) for k, v in t.items()} for t in targets]
boxes = targets[number]['boxes'].cpu().detach().numpy().astype(np.int32)
sample = images[number].permute(1,2,0).cpu().numpy()
fig, ax = plt.subplots(1, 1, figsize=(16, 8))

for box in boxes:
    cv2.rectangle(sample,
                  (box[0], box[1]),
                  (box[2], box[3]),
                  (0, 220, 0), 3)
    
ax.set_axis_off()
ax.imshow(sample)

In [ ]:
number = 0
images = [images_t[0].cpu()]
targets = clean_res(res, 0.6)

images = list(image.to(device) for image in images)
targets = [{k: v.to(device) for k, v in t.items()} for t in targets]
boxes = targets[number]['boxes'].cpu().detach().numpy().astype(np.int32)
sample = images[number].permute(1,2,0).cpu().numpy()
fig, ax = plt.subplots(1, 1, figsize=(16, 8))

for box in boxes:
    cv2.rectangle(sample,
                  (box[0], box[1]),
                  (box[2], box[3]),
                  (0, 220, 0), 3)
    
ax.set_axis_off()
ax.imshow(sample)

In [ ]:
# model.eval()
sample_sub = pd.read_csv('../input/siim-covid19-detection/sample_submission.csv')
test_path = '../input/siim-covid19-resized-to-256px-jpg/test/'

sample_sub = sample_sub.apply(lambda x:predict(x), axis = 1)

In [ ]:
sample_sub.to_csv('submission.csv')